# Fluvial-Pluvial Delineation

### Validation and Verification

**Delineation Quality Checks**:

1. **Hydraulic Connectivity**
   - Verify fluvial flooding is connected to river/stream
   - Check pluvial areas are isolated from channel
   - Validate flow path analysis

2. **Threshold Sensitivity**
   - Test multiple distance thresholds
   - Document threshold selection rationale
   - Check for reasonable transitions between classes

3. **Physical Plausibility**
   - Low areas far from channels should be pluvial
   - Areas adjacent to channels should be fluvial
   - Check for unrealistic classifications

4. **Comparison with Field Observations**
   - Known flooding sources (if available)
   - Historical flood extents
   - Local knowledge validation

**Output Verification**:
```python
# Check classification statistics
fluvial_area = (classified_raster == 1).sum() * pixel_area
pluvial_area = (classified_raster == 2).sum() * pixel_area
print(f"Fluvial: {fluvial_area / 1e6:.2f} km²")
print(f"Pluvial: {pluvial_area / 1e6:.2f} km²")
print(f"Ratio: {fluvial_area / pluvial_area:.2f}")
```

### Professional Review Requirements
- [ ] Methodology peer reviewed
- [ ] Threshold selection documented and justified
- [ ] Results compared with historical flooding patterns
- [ ] Uncertainty quantified and communicated
- [ ] Limitations clearly stated in deliverables
- [ ] Licensed professional certification (if regulatory use)

### Fluvial vs Pluvial Flooding

**Definitions**:
- **Fluvial**: River/stream overflow flooding (channel-driven)
- **Pluvial**: Surface water flooding from direct rainfall (not channel-driven)

**Why Delineation Matters**:
- Different mitigation strategies
- Different regulatory frameworks
- Different risk assessment methods
- Compound flooding may occur (both types simultaneously)

**This Notebook Demonstrates**:
- Spatial delineation using HEC-RAS 2D results
- Attribution based on flow paths and connectivity
- Visualization of flood source areas

### Reference
- [FEMA Flood Insurance Study Guidelines](https://www.fema.gov/flood-maps/guidance-partners/guidelines-standards)
- [Urban Drainage and Flood Control District Guidelines](https://udfcd.org/criteria-manual/)
- HEC-RAS 2D Modeling User's Manual (compound flooding chapter)

### Professional Context
Fluvial/pluvial delineation is critical for:
- Flood risk communication to stakeholders
- Insurance rating and premiums
- Infrastructure design (different protection levels)
- Regulatory compliance (separate standards may apply)

**Important**: This is a **demonstration workflow**. Professional delineation requires:
- Review by licensed professional engineer
- Validation against field observations
- Consideration of site-specific hydraulics
- Peer review of methodology

In [1]:
# =============================================================================
# DEVELOPMENT MODE TOGGLE
# =============================================================================
USE_LOCAL_SOURCE = False  # <-- TOGGLE THIS

if USE_LOCAL_SOURCE:
    import sys
    from pathlib import Path
    local_path = str(Path.cwd().parent)
    if local_path not in sys.path:
        sys.path.insert(0, local_path)
    print(f"📁 LOCAL SOURCE MODE: Loading from {local_path}/ras_commander")
else:
    print("📦 PIP PACKAGE MODE: Loading installed ras-commander")

# Import ras-commander
from ras_commander import HdfBase, HdfMesh, HdfResultsMesh, HdfResultsPlot, HdfUtils, RasCmdr, RasExamples, RasPlan, init_ras_project, ras

# Verify which version loaded
import ras_commander
print(f"✓ Loaded: {ras_commander.__file__}")

📦 PIP PACKAGE MODE: Loading installed ras-commander


✓ Loaded: c:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\ras_commander\__init__.py


## Parameters

Configure these values to customize the notebook for your project.

In [ ]:
# =============================================================================
# PARAMETERS - Edit these to customize the notebook
# =============================================================================
from pathlib import Path

# Project Configuration
PROJECT_NAME = "BaldEagleCrkMulti2D"           # Example project to extract
RAS_VERSION = "6.6"               # HEC-RAS version (6.3, 6.5, 6.6, etc.)

# Output Configuration
OUTPUTS_DIR = Path("_outputs") / "610_fluvial_pluvial_delineation"  # Artifacts saved here

# HDF Analysis Settings
PLAN = "06"                       # Plan number (for HDF file path)
TIME_INDEX = -1                   # Time step index (-1 = last)
PROFILE = "Max"                   # Profile name for steady analysis

# Create output directory
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)
print(f"Outputs will be saved to: {OUTPUTS_DIR.absolute()}")


# Delineate Fluvial and Pluvial Areas using RAS-Commander

We will leverage the HEC RAS Summary Outputs to delineate the Fluvial and Pluvial Areas

Maximum Water Surface Elevation (WSEL) for each cell is recorded, along with the timestamps of when the maximum WSEL occurs.

By locating adjacent cells with dissimilar timestamps, we can delineate the Fluvial and Pluvial Areas.




A note about datframe types: 

Information from the HEC-RAS plan files are generally dataframes.  The text file interface is for the 32-bit side of HEC-RAS and all spatial data is most easily accessed in the HDF files.  This includes plan_df, geom_df, hdf_paths_df

Geometry elements (Mesh Faces and Nodes) are provided as Geodataframes (cell_polygons_gdf, boundary_gdf)


## Package Installation and Environment Setup
Uncomment and run package installation commands if needed

In [2]:
# Install ras-commander from pip (uncomment to install if needed)
#!pip install ras-commander
# This installs ras-commander and all dependencies

In [3]:
# Import all required modules
#from ras_commander import HdfBase, HdfMesh, HdfResultsMesh, HdfResultsPlot, HdfUtils, RasCmdr, RasExamples, RasPlan, init_ras_project, ras  # Import all ras-commander modules

# Import the required libraries for this notebook
import h5py
import numpy as np
import requests
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
from shapely.geometry import Point, LineString, Polygon
import xarray as xr
from pathlib import Path

In [4]:
# Download the BaldEagleCrkMulti2D project from HEC and run plan 06

# Define the path to the BaldEagleCrkMulti2D project
import logging

# Extract project using suffix parameter (returns correct path)
bald_eagle_path = RasExamples.extract_project(PROJECT_NAME, suffix="14")

# Initialize the RAS project using the default global ras object
init_ras_project(bald_eagle_path, RAS_VERSION)
logging.info(f"Bald Eagle project initialized with folder: {ras.project_folder}")

# Define the plan number to execute
plan_number = plan_number

# Update the run flags in the plan file
RasPlan.update_run_flags(
    plan_number,
    geometry_preprocessor=True,  # Run HTab
    unsteady_flow_simulation=True,  # Run UNet
    post_processor=True,  # Run PostProcess
    floodplain_mapping=False,  # Run RASMapper
)

# Execute Plan 06 using RasCmdr with skip_existing=True
# This will skip execution if HDF results already exist with "Complete Process"
print(f"Executing Plan {plan_number} for the Bald Eagle Creek project...")
success_bald_eagle = RasCmdr.compute_plan(plan_number, skip_existing=True)
if success_bald_eagle:
    print(f"Plan {plan_number} executed successfully (or skipped - results exist).")
else:
    print(f"Plan {plan_number} execution failed for Bald Eagle.")


2025-12-16 17:14:38 - ras_commander.RasExamples - INFO - Found zip file: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\Example_Projects_6_6.zip


2025-12-16 17:14:38 - ras_commander.RasExamples - INFO - Loading project data from CSV...


2025-12-16 17:14:38 - ras_commander.RasExamples - INFO - Loaded 68 projects from CSV.


2025-12-16 17:14:38 - ras_commander.RasExamples - INFO - ----- RasExamples Extracting Project -----


2025-12-16 17:14:38 - ras_commander.RasExamples - INFO - Extracting project 'BaldEagleCrkMulti2D' as 'BaldEagleCrkMulti2D_14'


...

2025-12-16 17:14:41 - ras_commander.RasCmdr - INFO - Using ras_object with project folder: C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_14


2025-12-16 17:14:41 - ras_commander.RasCmdr - INFO - Running HEC-RAS from the Command Line:


2025-12-16 17:14:41 - ras_commander.RasCmdr - INFO - Running command: "C:\Program Files (x86)\HEC\HEC-RAS\6.6\Ras.exe" -c "C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_14\BaldEagleDamBrk.prj" "C:\Users\billk_clb\anaconda3\envs\rascmdr_piptest\Lib\site-packages\examples\example_projects\BaldEagleCrkMulti2D_14\BaldEagleDamBrk.p06"


Executing Plan 06 for the Bald Eagle Creek project...


KeyboardInterrupt: 

###  OPTIONAL: Use your own project instead

your_project_path = Path(r"D:\yourprojectpath")

init_ras_project(your_project_path, "6.6")
plan_number = "01"  # Plan number to use for this notebook 



### If you use this code cell, don't run the previous cell or change to markdown
### NOTE: Ensure the HDF Results file was generated by HEC-RAS Version 6.x or above

# Explore Project Dataframes using 'ras' Object

In [5]:
# Load Plan and Geometry Dataframes and find Plan and Geometry HDF Paths

# Display plan_df for bald_eagle project
print("Plan DataFrame for bald_eagle project:")
ras.plan_df

In [6]:
print("\nGeometry DataFrame for the project:")
ras.geom_df

In [7]:
print("\nUnsteady DataFrame for the project:")
ras.unsteady_df

In [8]:
print("\nBoundary Conditions DataFrame for the project:")
ras.boundaries_df 

# Find Paths for Results and Geometry HDF's

In [9]:
plan_number = plan_number
print(plan_number)

# Get the plan HDF path for the plan_number defined above
plan_hdf_path = ras.plan_df.loc[ras.plan_df['plan_number'] == plan_number, 'HDF_Results_Path'].values[0]

In [10]:
plan_hdf_path

In [11]:
# Get the geometry HDF path
geom_hdf_path = ras.plan_df.loc[ras.plan_df['plan_number'] == plan_number, 'Geom Path'].values[0] + '.hdf'

In [12]:
geom_hdf_path


In [13]:
print(f"\nPlan HDF path for Plan {plan_number}: {plan_hdf_path}")
print(f"Geometry HDF path for Plan {plan_number}: {geom_hdf_path}")

# Fluvial Pluvial Delineation using RAS-Commander 

Using the Maximum WSE Results layer, which contains the maximum water surface and time stamp of the maximum water surface, mesh cell faces are categorized.  If the difference in time (delta_t) in hours is greater than the (user defined, default 12) duration specified, that mesh cell face is added to the fluvial-pluvial boundary dataset. 

This is meant to provide a draft fluvial-pluvial boundary for floodplain analysis, to the extent it can be derived directly from the HEC-RAS results files. 

The function attempts to combine adjacent line segments to simplify the resulting geometry, but GIS cleanup and manual interpolation will be required to create a closed polygon boundary that could be used for further processing steps.  However, this approach does provide an efficient method for providing a draft boundary that is based on HEC-RAS's direct computations and mesh cell faces. 

In [14]:
# Using mesh_max_ws, get the cell coordinates and plot the max water surface as a map
import matplotlib.pyplot as plt
from ras_commander import HdfMesh
from ras_commander import HdfResultsMesh
from shapely.geometry import Point

# Get mesh max water surface
max_ws_df = HdfResultsMesh.get_mesh_max_ws(plan_hdf_path)

print("max_ws_df")
print(max_ws_df)

# If you get an error here, you may have a pre-6.0 HDF.  Re-run in 6.x to generate a new results file

In [15]:
# Call the function to plot
HdfResultsPlot.plot_results_max_wsel(max_ws_df)

# Plot the time of maximum water surface elevation
HdfResultsPlot.plot_results_max_wsel_time(max_ws_df)

# Print the first few rows of the merged dataframe for verification
print("\nFirst few rows of the merged dataframe:")
max_ws_df


In [16]:
# Use HdfUtils for extracting projection
print("\nExtracting Projection from HDF")
projection = HdfBase.get_projection(hdf_path=geom_hdf_path)
if projection:
    print(f"Projection: {projection}")
else:
    print("No projection information found.")

In [17]:
# Example: Extract Cell Polygons
print("\nExample 6: Extracting Cell Polygons")
cell_polygons_gdf = HdfMesh.get_mesh_cell_polygons(geom_hdf_path)


# Call the function to plot cell polygons
#cell_polygons_gdf = HdfFluvialPluvial.plot_cell_polygons(cell_polygons_gdf, projection)


In [18]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, Polygon, MultiLineString
from collections import defaultdict
from tqdm import tqdm
from rtree import index


# Example usage:
boundary_gdf = HdfFluvialPluvial.calculate_fluvial_pluvial_boundary(plan_hdf_path, delta_t=72)

In [19]:
# Calculate statistics about the boundary line lengths
boundary_lengths = boundary_gdf.geometry.length

print("Boundary line length statistics:")
print(f"Max length: {boundary_lengths.max():.2f}")
print(f"Min length: {boundary_lengths.min():.2f}")
print(f"Average length: {boundary_lengths.mean():.2f}")
print(f"Median length: {boundary_lengths.median():.2f}")

# Print general information about the boundary GeoDataFrame
print("\nBoundary GeoDataFrame info:")
print(boundary_gdf.info())

In [20]:
# Visualize the results
fig, ax = plt.subplots(figsize=(12, 8))
cell_polygons_gdf.plot(ax=ax, edgecolor='gray', facecolor='none', alpha=0.5)
boundary_gdf.plot(ax=ax, color='red', linewidth=2)
plt.title('Fluvial-Pluvial Boundary')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.show()

In [21]:
length_threshold = 3000 #in same units as X and Y coordinates

# Filter out boundary lines below the length threshold
filtered_boundary_gdf = boundary_gdf[boundary_lengths >= length_threshold]
highlighted_boundary_gdf = boundary_gdf[boundary_lengths < length_threshold]

# Visualize the results with highlighted boundaries below the threshold
fig, ax = plt.subplots(figsize=(12, 8))
cell_polygons_gdf.plot(ax=ax, edgecolor='gray', facecolor='none', alpha=0.5)
filtered_boundary_gdf.plot(ax=ax, color='red', linewidth=2, label='Valid Boundaries')
highlighted_boundary_gdf.plot(ax=ax, color='blue', linewidth=2, linestyle='--', label='Highlighted Boundaries Below Threshold')
plt.title('Fluvial-Pluvial Boundary with Length Threshold')
plt.xlabel('X Coordinate')
plt.ylabel('Y Coordinate')
plt.legend()
plt.show()


In [22]:
# Example usage using Optional min_line_length argument:
boundary_gdf = HdfFluvialPluvial.calculate_fluvial_pluvial_boundary(plan_hdf_path, delta_t=72, min_line_length=3000)

In [23]:
# To get the classified flood zones as polygons:
flood_polygons_gdf = HdfFluvialPluvial.generate_fluvial_pluvial_polygons(
    plan_hdf_path, 
    delta_t=10, 
    temporal_tolerance_hours=1.0,
    min_polygon_area_acres=1000
)

flood_polygons_gdf

# Plot the classified flood zones as polygons, colored by classification
fig, ax = plt.subplots(figsize=(12, 8))
flood_polygons_gdf.plot(
    ax=ax,
    column="classification",
    categorical=True,
    legend=True,
    edgecolor="black",
    alpha=0.7
)
plt.title("Fluvial-Pluvial Classified Flood Zones")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()


In [24]:
# To get the classified flood zones as polygons:
flood_polygons_gdf = HdfFluvialPluvial.generate_fluvial_pluvial_polygons(
    plan_hdf_path, 
    delta_t=12, 
    temporal_tolerance_hours=1.0,
    min_polygon_area_acres=200
)

flood_polygons_gdf

# Plot the classified flood zones as polygons, colored by classification
fig, ax = plt.subplots(figsize=(12, 8))
flood_polygons_gdf.plot(
    ax=ax,
    column="classification",
    categorical=True,
    legend=True,
    edgecolor="black",
    alpha=0.7
)
plt.title("Fluvial-Pluvial Classified Flood Zones")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()


In [25]:
# To get the classified flood zones as polygons:
flood_polygons_gdf = HdfFluvialPluvial.generate_fluvial_pluvial_polygons(
    plan_hdf_path, 
    delta_t=14, 
    temporal_tolerance_hours=1.0,
    min_polygon_area_acres=1000
)

flood_polygons_gdf

# Plot the classified flood zones as polygons, colored by classification
fig, ax = plt.subplots(figsize=(12, 8))
flood_polygons_gdf.plot(
    ax=ax,
    column="classification",
    categorical=True,
    legend=True,
    edgecolor="black",
    alpha=0.7
)
plt.title("Fluvial-Pluvial Classified Flood Zones")
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()


In [26]:
flood_polygons_gdf.to_file(driver='GeoJSON', filename='flood_polygons.geojson')

In [27]:
# Create fluvial_pluvial_boundary subfolder in project directory
output_dir = ras.project_folder / "fluvial_pluvial_boundary"
output_dir.mkdir(exist_ok=True)
print(f"Output directory created/verified at: {output_dir}")

# Save to GeoJSON in output directory
boundary_gdf.to_file(output_dir / 'fluvial_pluvial_boundary.geojson', driver='GeoJSON')